# Defense

Is it typically the case that the color classes in a NAC-coloring are balanced with respect to the two colors (especially when the graph has only few NAC-colorings)?
Or does it often happen (in practice) that there exists some NAC-coloring that is largely biased towards one of the two colors?

In [1]:
from typing import *
from dataclasses import dataclass
from collections import defaultdict, deque
import random
import importlib
from random import Random
from enum import Enum

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline as backend_inline
from matplotlib.backends import backend_agg
from matplotlib.figure import Figure
from matplotlib.ticker import MaxNLocator

import numpy as np
import pandas as pd
import networkx as nx
import os
import time
import datetime
import signal
import itertools
import base64

from tqdm import tqdm

import nac as nac
from nac import MonochromaticClassType

import benchmarks
from benchmarks import datasets
from benchmarks import generators
import benchmarks.notebook_utils
from benchmarks.notebook_utils import *

seed=42
TEST=False
BENCHMARKS=False
ANALYTICS=True
SEARCH=False


In [ ]:
importlib.reload(nac)
importlib.reload(benchmarks.datasets)
importlib.reload(benchmarks.generators)
importlib.reload(benchmarks)
importlib.reload(benchmarks.notebook_utils)

_BENCH_FILE_START_V2 = "bench_res_v2"
_BENCH_FILE_START_V3 = "bench_res_v3"
_BENCH_FILE_START_V4 = "bench_res_v4"

OUTPUT_DIR = os.path.join("benchmarks", "runs")
os.makedirs(OUTPUT_DIR, exist_ok=True)

benchmarks.notebook_utils.OUTPUT_DIR = OUTPUT_DIR
benchmarks.notebook_utils.OUTPUT_BENCH_FILE_START = _BENCH_FILE_START_V4
benchmarks.notebook_utils.OUTPUT_VERBOSE = False

In [ ]:
from nac import NAC_colorings, canonical_NAC_coloring
import numpy as np

In [ ]:
df = load_records()

In [ ]:
def evaluate(df: pd.DataFrame, sample_size: int = 1000, random_state: int = 42) -> dict[nx.Graph, list[Tuple[Container, Container]]]:
    df = df.sample(sample_size, random_state=random_state)
    res = dict()
    for graph in tqdm(df['graph']):
        graph = graph_from_id(graph)
        res[graph] = list(NAC_colorings(graph))
    return res

In [ ]:
def analyze(data: list[list[tuple[Container, Container]]]) -> pd.DataFrame:
    ratios: list[tuple[float, float]] = []
    for record in data:
        colorings = set(canonical_NAC_coloring(col) for col in record)
        item_weight = 1/len(colorings)
        for col in colorings:
            if len(col[0]) > len(col[1]):
                col = (col[1], col[0])
            ratios.append((len(col[0]) / len(col[1]), item_weight))
    df = pd.DataFrame(ratios, columns=["ratio", "weight"])
    df.sort_values(by="ratio", inplace=True)
    return df

In [ ]:
def showcase(df: pd.DataFrame) -> None:
    print("Count:", len(df))
    display(df["ratio"].describe())
    print("Weighted average:", np.average(df["ratio"], weights=df["weight"]))
    df["ratio"].hist(bins=20)

## Minimally rigid

In [ ]:
min_rigid = df.query("dataset == 'minimally_rigid_random' and nac_all_coloring_no > 0 and vertex_no >= 18")
len(min_rigid)

In [ ]:
min_rigid_colorings = evaluate(min_rigid)

In [ ]:
min_ser = analyze(min_rigid_colorings.values())

In [ ]:
showcase(min_ser)

## Globally rigid

### Single NAC-coloring

In [ ]:
glob_rigid_single = df.query("dataset == 'globally_rigid' and nac_all_coloring_no > 0 and vertex_no >= 18 and nac_all_coloring_no <= 2")
len(glob_rigid_single)

In [ ]:
glob_rigid_single_colorings = evaluate(glob_rigid_single)

In [ ]:
glob_single_ser = analyze(glob_rigid_single_colorings.values())

In [ ]:
showcase(glob_single_ser)

### Few NAC-coloring

In [ ]:
glob_rigid_few = df.query("dataset == 'globally_rigid' and nac_all_coloring_no > 0 and vertex_no >= 18 and nac_all_coloring_no <= 20")
len(glob_rigid_few)

In [ ]:
glob_rigid_few_colorings = evaluate(glob_rigid_few)

In [ ]:
glob_few_ser = analyze(glob_rigid_few_colorings.values())

In [ ]:
showcase(glob_few_ser)